# Data processing -- stale experiments

In [23]:
using Tidier

In [ ]:
using DBnomics
natl_acc = rdb("Eurostat", "nama_10_gdp", mask="B1GQ")

In [ ]:
using HTTP
res = HTTP.get("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true")

In [ ]:
import Downloads
using CSV
natl_acc = CSV.read(Downloads.download("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true"), DataFrame)

In [ ]:
using CSV
natl_acc = CSV.read("datasets/estat_nama_10_gdp.tsv.gz", DataFrame; delim="\t")

In [ ]:
natl_acc = read_csv("datasets/estat_nama_10_gdp.tsv.gz")

In [ ]:
natl_acc = DataFrames.rename(natl_acc, "freq,unit,na_item,geo\\TIME_PERIOD" => "mixed")

In [ ]:
nat_acc = @chain natl_acc begin
    @separate(mixed, (freq,unit,na_item,geo), ",")
    @pivot_longer(-(freq:geo), names_to=year,values_to=value)
    @select(-freq)
    @mutate(value = as_float(value), year = as_integer(str_squish(year)))
end

In [ ]:
bg_nat_acc = @chain nat_acc begin
    @filter(geo == "BG", unit ==  "CLV10_MNAC", na_item in ("B1GQ", "P51G"))
    @select(na_item,year,value)
    @drop_missing()
end

In [ ]:
# names(nat_acc)
println(@distinct(nat_acc.na_item))

## To explore

FredData -- https://juliahub.com/ui/Packages/General/FredData
SDMX -- https://juliahub.com/ui/Packages/General/SDMX
IMFData -- https://juliahub.com/ui/Packages/General/IMFData


In [27]:
using HTTP
using JSON
using DataFrames
using IterTools

function get_eurostat_from_URL(url::String)
    # Make HTTP request
    response = HTTP.get(url)
    
    if response.status == 200
        # Parse JSON response
        data = JSON.parse(String(response.body))
        
        # Extract dimensions, IDs, and sizes
        dimensions = data["dimension"]
        id_list = data["id"]  # Order of dimensions
        size = data["size"]  # Size of each dimension
        
        # Create mapping for each dimension
        mappings = Dict(dim => Dict(v => k for (k, v) in dimensions[dim]["category"]["index"]) for dim in id_list)
        
        # Generate Cartesian product of indices
        ranges = [1:s for s in size]  # Julia uses 1-based indexing
        cartesian_product = collect(product(ranges...))
        
        # Decode indices into human-readable labels
        decoded_rows = [
            Dict(dim => mappings[dim][string(index - 1)] for (dim, index) in zip(id_list, row)) 
            for row in cartesian_product
        ]
        
        # Add 'Value' column from data['value']
        for (i, row) in enumerate(decoded_rows)
            row["Value"] = get(data["value"], string(i - 1), missing)  # Default to `missing` if value is not found
        end
        
        # Convert to DataFrame
        return DataFrame(decoded_rows)
    else
        println("Failed to retrieve data: $(response.status)")
        return nothing
    end
end


get_eurostat_from_URL (generic function with 1 method)

In [29]:
df = get_eurostat_from_URL("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/namq_10_gdp?format=JSON&geo=BG&na_item=B1GQ&lang=en")

LoadError: KeyError: key "0" not found